Apply the R scripts to connect TF-regulons with Stroke DE genes.

In [2]:
# The following script will be used with MAST package for DE calculation.

# DE-centered analysis:


# Summary of Workflow:

# Identify DEGs: Use FindMarkers() to get DEGs for D2 and D14 endothelial cells vs. Sham.

# Cluster DEGs into gene sets: Use GO/KEGG enrichment (clusterProfiler) or gene clustering (GSVA).

# Identify differentially active TFs: Use the TF assay in Seurat.

# Match DEGs to TFs: Check which DEGs belong to known TF regulons.

# Perform enrichment analysis for TFs: Use Fisher’s exact test to find core TFs.

# Visualize results: Use heatmaps (pheatmap) and networks (igraph).

##############################################################################

# Step 1: Identify Differentially Expressed Genes (DEGs)

# We need to compare D2 and D14 endothelial cells to control (sham) endothelial cells.

# Subset Endothelial Cells:

# library(Seurat)
# # Extract endothelial cells
# endo_cells <- subset(seurat_obj, idents = "Endothelial")  # Adjust based on annotation

# Find DEGs (e.g., D2 vs. Sham)


# endo_cells$group <- factor(endo_cells$group, levels = c("Sham", "D2", "D14"))
# deg_d2 <- FindMarkers(endo_cells, ident.1 = "D2", ident.2 = "Sham", test.use = "DESeq2")
# deg_d14 <- FindMarkers(endo_cells, ident.1 = "D14", ident.2 = "Sham", test.use = "DESeq2")


# Filter significant genes (adjusted p-value < 0.05 & log2FC threshold)

# deg_d2_sig <- deg_d2 %>% dplyr::filter(p_val_adj < 0.05 & abs(log2FC) > 0.25)
# deg_d14_sig <- deg_d14 %>% dplyr::filter(p_val_adj < 0.05 & abs(log2FC) > 0.25)

# Step 2: Identify Gene Sets (Modules) Characterizing D2 and D14 Endothelial Cells

# You can use clustering methods such as WGCNA, k-means, or functional enrichment analysis.

# Functional Enrichment Analysis (GO/KEGG)

# library(clusterProfiler)

# # Convert gene symbols
# gene_list_d2 <- deg_d2_sig$gene
# gene_list_d14 <- deg_d14_sig$gene

# # Enrichment analysis
# go_d2 <- enrichGO(gene_list_d2, OrgDb = org.Mm.eg.db, keyType = "SYMBOL", ont = "BP")
# go_d14 <- enrichGO(gene_list_d14, OrgDb = org.Mm.eg.db, keyType = "SYMBOL", ont = "BP")

# # Visualize
# dotplot(go_d2) + ggtitle("GO Enrichment - D2 Endothelial")
# dotplot(go_d14) + ggtitle("GO Enrichment - D14 Endothelial")

# Gene Set Clustering

# library(GSEABase)
# library(GSVA)

# gsva_scores <- gsva(assay(seurat_obj, "RNA"), gene_sets, method = "ssgsea")
# Cluster DEGs into functional modules using k-means or hierarchical clustering.

# Step 3: Connect Gene Sets to TFs

# Now, we will use the TF assay (AUC scores) and regulon list.

# Check if TFs Are Differentially Active

# tf_activity <- as.data.frame(seurat_obj@assays$TF@data)  # Extract AUC scores

# # Find differentially activated TFs
# tf_deg_d2 <- FindMarkers(seurat_obj, ident.1 = "D2", ident.2 = "Sham", assay = "TF")
# tf_deg_d14 <- FindMarkers(seurat_obj, ident.1 = "D14", ident.2 = "Sham", assay = "TF")

# Match TFs to DEGs via Regulons

# # Get regulons from the list
# tf_regulon_d2 <- lapply(names(tf_regulons), function(tf) {
#   genes <- tf_regulons[[tf]]
#   genes[genes %in% gene_list_d2]  # Keep only DEGs
# })

# tf_regulon_d14 <- lapply(names(tf_regulons), function(tf) {
#   genes <- tf_regulons[[tf]]
#   genes[genes %in% gene_list_d14]  # Keep only DEGs
# })

# Identify Core TFs

# #  Instead of Ranking TFs Using GSVA Pathway Scores (by correlating TF activity [AUC scores] with GSVA scores),

# # we use DEGs + Fisher’s Exact Test,

# Rank TFs based on their overlap with DEGs.

# Use Fisher’s exact test to test for enrichment.


# fisher_test <- function(tf, gene_list) {
#   genes_in_tf <- tf_regulons[[tf]]
#   overlap <- length(intersect(genes_in_tf, gene_list))
#   contingency <- matrix(c(overlap, length(genes_in_tf) - overlap, 
#                           length(gene_list) - overlap, 
#                           length(universe_genes) - length(genes_in_tf) - length(gene_list) + overlap), 
#                         nrow = 2)
#   fisher.test(contingency)$p.value
# }


# tf_scores_d2 <- sapply(names(tf_regulons), fisher_test, gene_list = gene_list_d2)
# tf_scores_d14 <- sapply(names(tf_regulons), fisher_test, gene_list = gene_list_d14)

# Adjust for multiple testing

# tf_scores_d2_adj <- p.adjust(tf_scores_d2, method = "BH")
# tf_scores_d14_adj <- p.adjust(tf_scores_d14, method = "BH")

# # Select significant TFs
# core_tfs_d2 <- names(tf_scores_d2_adj[tf_scores_d2_adj < 0.05])
# core_tfs_d14 <- names(tf_scores_d14_adj[tf_scores_d14_adj < 0.05])

# Step 4: Visualize TF-Gene Relationships

# Heatmap of Core TF Activity

# library(pheatmap)

# tf_activity_core_d2 <- tf_activity[core_tfs_d2, ]
# pheatmap(tf_activity_core_d2, scale = "row", cluster_rows = TRUE, cluster_cols = TRUE)
# Network Visualization (TF-Gene Associations)

# library(igraph)

# edges_d2 <- data.frame(TF = rep(core_tfs_d2, sapply(tf_regulon_d2, length)), 
#                        Gene = unlist(tf_regulon_d2))

# graph_d2 <- graph_from_data_frame(edges_d2, directed = FALSE)
# plot(graph_d2, vertex.size = 5, edge.arrow.size = 0.5)

Same Workflow as splitted into code blocks:

#####################################################################################

DE-centered analysis:

Summary of Workflow:

Identify DEGs: Use FindMarkers() to get DEGs for D2 and D14 endothelial cells vs. Sham.

Cluster DEGs into gene sets: Use GO/KEGG enrichment (clusterProfiler) or gene clustering (GSVA).

Identify differentially active TFs: Use the TF assay in Seurat.

Match DEGs to TFs: Check which DEGs belong to known TF regulons.

Perform enrichment analysis for TFs: Use Fisher’s exact test to find core TFs.

Visualize results: Use heatmaps (pheatmap) and networks (igraph).

##############################################################################

Step 1: Identify Differentially Expressed Genes (DEGs)

In [ ]:
# We need to compare D2 and D14 endothelial cells to control (sham) endothelial cells.

# Subset Endothelial Cells:

library(Seurat)
# Extract endothelial cells
endo_cells <- subset(seurat_obj, idents = "Endothelial")  # Adjust based on annotation

In [ ]:
# Find DEGs (e.g., D2 vs. Sham)

endo_cells$group <- factor(endo_cells$group, levels = c("Sham", "D2", "D14"))
deg_d2 <- FindMarkers(endo_cells, ident.1 = "D2", ident.2 = "Sham", test.use = "DESeq2") # use MAST instead.
deg_d14 <- FindMarkers(endo_cells, ident.1 = "D14", ident.2 = "Sham", test.use = "DESeq2") # use MAST instead.

In [ ]:
# Filter significant genes (adjusted p-value < 0.05 & log2FC threshold)

deg_d2_sig <- deg_d2 %>% dplyr::filter(p_val_adj < 0.05 & abs(log2FC) > 0.25)
deg_d14_sig <- deg_d14 %>% dplyr::filter(p_val_adj < 0.05 & abs(log2FC) > 0.25)

Step 2: Identify Gene Sets (Modules) Characterizing D2 and D14 Endothelial Cells

You can use clustering methods such as WGCNA, k-means, or functional enrichment analysis.

Functional Enrichment Analysis (GO/KEGG)

In [ ]:
library(clusterProfiler)

# Convert gene symbols
gene_list_d2 <- deg_d2_sig$gene
gene_list_d14 <- deg_d14_sig$gene

# Enrichment analysis
go_d2 <- enrichGO(gene_list_d2, OrgDb = org.Mm.eg.db, keyType = "SYMBOL", ont = "BP")
go_d14 <- enrichGO(gene_list_d14, OrgDb = org.Mm.eg.db, keyType = "SYMBOL", ont = "BP")

# Visualize
dotplot(go_d2) + ggtitle("GO Enrichment - D2 Endothelial")
dotplot(go_d14) + ggtitle("GO Enrichment - D14 Endothelial")

In [ ]:
# Gene Set Clustering

library(GSEABase)
library(GSVA)

gsva_scores <- gsva(assay(seurat_obj, "RNA"), gene_sets, method = "ssgsea")

# Important Note: We can also cluster DEGs into functional modules using k-means or hierarchical clustering.

In [ ]:
Step 3: Connect Gene Sets to TFs

In [ ]:
Now, we will use the TF assay (AUC scores) and regulon list.

Check if TFs Are Differentially Active

tf_activity <- as.data.frame(seurat_obj@assays$TF@data)  # Extract AUC scores

# Find differentially activated TFs
tf_deg_d2 <- FindMarkers(seurat_obj, ident.1 = "D2", ident.2 = "Sham", assay = "TF")
tf_deg_d14 <- FindMarkers(seurat_obj, ident.1 = "D14", ident.2 = "Sham", assay = "TF")

Match TFs to DEGs via Regulons

# Get regulons from the list
tf_regulon_d2 <- lapply(names(tf_regulons), function(tf) {
  genes <- tf_regulons[[tf]]
  genes[genes %in% gene_list_d2]  # Keep only DEGs
})

tf_regulon_d14 <- lapply(names(tf_regulons), function(tf) {
  genes <- tf_regulons[[tf]]
  genes[genes %in% gene_list_d14]  # Keep only DEGs
})

In [ ]:
Identify Core TFs

#  Instead of Ranking TFs Using GSVA Pathway Scores (by correlating TF activity [AUC scores] with GSVA scores),

# we use DEGs + Fisher’s Exact Test,

Rank TFs based on their overlap with DEGs.

In [ ]:
Use Fisher’s exact test to test for enrichment.


fisher_test <- function(tf, gene_list) {
  genes_in_tf <- tf_regulons[[tf]]
  overlap <- length(intersect(genes_in_tf, gene_list))
  contingency <- matrix(c(overlap, length(genes_in_tf) - overlap, 
                          length(gene_list) - overlap, 
                          length(universe_genes) - length(genes_in_tf) - length(gene_list) + overlap), 
                        nrow = 2)
  fisher.test(contingency)$p.value
}


tf_scores_d2 <- sapply(names(tf_regulons), fisher_test, gene_list = gene_list_d2)
tf_scores_d14 <- sapply(names(tf_regulons), fisher_test, gene_list = gene_list_d14)

Adjust for multiple testing

tf_scores_d2_adj <- p.adjust(tf_scores_d2, method = "BH")
tf_scores_d14_adj <- p.adjust(tf_scores_d14, method = "BH")

# Select significant TFs
core_tfs_d2 <- names(tf_scores_d2_adj[tf_scores_d2_adj < 0.05])
core_tfs_d14 <- names(tf_scores_d14_adj[tf_scores_d14_adj < 0.05])

In [ ]:
Step 4: Visualize TF-Gene Relationships

In [ ]:
Heatmap of Core TF Activity

library(pheatmap)

tf_activity_core_d2 <- tf_activity[core_tfs_d2, ]
pheatmap(tf_activity_core_d2, scale = "row", cluster_rows = TRUE, cluster_cols = TRUE)
Network Visualization (TF-Gene Associations)

library(igraph)

edges_d2 <- data.frame(TF = rep(core_tfs_d2, sapply(tf_regulon_d2, length)), 
                       Gene = unlist(tf_regulon_d2))

graph_d2 <- graph_from_data_frame(edges_d2, directed = FALSE)
plot(graph_d2, vertex.size = 5, edge.arrow.size = 0.5)